In [1]:
import itertools
import matlab.engine
import plotly.express as px

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import time
import pandas as pd
import glob
from copy import deepcopy
import os
import sys
from joblib import Parallel, delayed, cpu_count, dump

project_path = r'D:\OneDrive - Nexus365\Project\pirate_fmri\Analysis'
sys.path.append(os.path.join(project_path,'scripts'))
from multivariate.dataloader import ActivityPatternDataLoader
from multivariate.helper import compute_rdm, checkdir, scale_feature,lower_tri
from multivariate.modelrdms import ModelRDM
from multivariate.rsa_searchlight import RSASearchLight
from multivariate.rsa_estimator import PatternCorrelation,MultipleRDMRegression,NeuralDirectionCosineSimilarity
from multivariate.rsa_runner import RSARunner

from sklearn.manifold import MDS,TSNE
import sklearn.manifold as manifold
from sklearn.decomposition import PCA
import scipy
import sklearn

In [2]:
#https://stackoverflow.com/questions/27426668/row-titles-for-matplotlib-subplot
from matplotlib.gridspec import SubplotSpec

def create_subtitle(fig: plt.Figure, grid: SubplotSpec, title: str):
    "Sign sets of subplots with title"
    row = fig.add_subplot(grid)
    # the '\n' is important
    row.set_title(f'{title}\n', fontweight='semibold')
    # hide subplot
    row.set_frame_on(False)
    row.axis('off')

In [3]:
with open(os.path.join(project_path,'scripts','pirate_defaults.json')) as f:
    pirate_defaults = json.load(f)
    subid_list = pirate_defaults['participants']['validids']
    fmribeh_dir = pirate_defaults['directory']['fmribehavior']
    fmridata_dir = pirate_defaults['directory']['fmri_data']
    nonlearners = pirate_defaults['participants']['nonlearnerids']
    nongeneralizers = pirate_defaults['participants']['nongeneralizerids']
    
ROIRSA_output_path = os.path.join(fmridata_dir,'ROIRSA','funcroi_respcompete_locwrtcetrefeature')
checkdir(ROIRSA_output_path)

preprocess = ["unsmoothedLSA","smoothed5mmLSA"]
n_sess = {
          "localizer":1,
          "noconcatall":1,
          "oddeven":2,
          "fourruns":4,
          }    


In [5]:
# maskdir = r'D:\OneDrive - Nexus365\Project\pirate_fmri\Analysis\data\fmri\masks\anat'
# with open(os.path.join(project_path,'scripts','anatomical_masks.json')) as f:    
#     anat_roi = list(json.load(f).keys())
# laterality = ["left","right","bilateral"]
# roi_dict = dict(zip(
#     ["frontal","ofc","hippocampus","parahippocampus","parietal"],
#     ["frontal","ofc","hippocampus","parahippocampus","parietal"]
#     ))

maskdir = r'D:\OneDrive - Nexus365\Project\pirate_fmri\Analysis\data\fmri\masks\func\wbsearch_reg_respcompete_locwrtcetrefeature_combinexy'
laterality = ["bilateral"]
roi_dict = dict(zip(
    ['localxy - L Cuneus','localxy - L Mid Cingulate',
     'localxy - R Mid Frontal','localxy - R Sup Parietal', 
     'localxy - Sup SuppMotor', 
     'global - Calcarine',
     ],
    ['xydist_LeftCuneus','xydist_RightCingulateMid',
     'xydist_RightFrontalMid','xydist_RightParietalSup',
     'xydist_SuppMotor','xysign_Calcarine'
     ]
     ))

# maskdir = r'D:\OneDrive - Nexus365\Project\pirate_fmri\Analysis\data\fmri\masks\func\wbsearch_reg_hierarchicalwrtcentre_combinexy'
# laterality = ["bilateral"]
# roi_dict = dict(zip(
#     ['localxy - Cuneus','localxy - L Mid Frontal',
#      'localxy - R Mid Frontal','localxy - Sup Parietal', 
#      'global - Calcarine',
#      ],
#     ['xydist_Cuneus','xydist_LeftFrontalMid_sphere',
#      'xydist_RightFrontalMid_sphere','xydist_ParietalSupL',
#      'xysign_Calcarine'
#      ]     
#             ))

In [ ]:
corr_rdm_names = [
                    "feature2d",
                    "gtlocEuclidean",
                    "global_xysign",
                    "locwrtcentre_localxy",
                    "locwrtlrbu_localxy",
                    "respglobal_xysign",
                    "resplocwrtcentre_localxy"
                 ]
modelrdm_cols_oldname = corr_rdm_names
modelrdm_cols_newname = modelrdm_cols_oldname
modelrdm_cols = dict(zip(modelrdm_cols_oldname,modelrdm_cols_newname))

for p in preprocess[:1]:
    beta_dir = {
        "fourruns":[os.path.join(fmridata_dir,p,'LSA_stimuli_navigation')],
        "noconcatall":[os.path.join(fmridata_dir,p,'LSA_stimuli_navigation')],
#        "localizer":[os.path.join(fmridata_dir,p,'LSA_stimuli_localizer')],
#        "oddeven":[os.path.join(fmridata_dir,p,'LSA_stimuli_navigation')]*2,

        }
    beta_fname = {
        "fourruns":['stimuli_4r.nii'],
        "localizer":['stimuli_1r.nii'],
        "noconcatall":['stimuli_all.nii'],
        "oddeven":['stimuli_odd.nii',
                   'stimuli_even.nii'],
        
        }
    vs_dir = {
              "no_selection":[],
              "reliability_ths0":[os.path.join(fmridata_dir,p,'reliability_noconcat')],
              "perm_rmask":[os.path.join(fmridata_dir,p,'reliability_noconcat')],
              }
    for ds_name,ds in beta_dir.items():
        for vselect,vdir in vs_dir.items():
            df = pd.read_csv(os.path.join(ROIRSA_output_path,f'roirdm_nocentering_{p}_{ds_name}_{vselect}.csv'))
            sub_type_dict = {'l':{1:'nonLearner',0:'Learner'},
                            'g':{1:'nonGeneralizer',0:'Generalizer'}}
            df['type'] = df['subid'].apply(
                lambda x: f"{sub_type_dict['g'][int(x in nongeneralizers)]}" #{sub_type_dict['l'][int(x in nonlearners)]} - 
            )
            df['run_comb'] = 1*(df['runA'] == df['runB'])
            df['run_comb'] = df['run_comb'].map({0:"between run",1:"within run"})
            df['group_comb'] = 1*(df['groupA'] == df['groupB'])
            df['group_comb'] = df['group_comb'].map({0:"between group",1:"within group"})
            #df = df[((df['groupA']==0)&(df['groupB']==0))]
            #df = df[(df['run_comb']=="between run")]
            for m,mn in modelrdm_cols.items():
                if m in df.columns :
                    fig_suptitle = f'{p} - {ds_name} - {vselect} - {mn}'
                    print(fig_suptitle)
                    plt.ioff()
                    prows, pcols = np.unique(df['run_comb']).size, len(roi_dict)
                    fig,axes = plt.subplots(nrows=prows,ncols=pcols,figsize=(60, 20))
                    j = 0
                    for _,(g_name,g_df) in list(enumerate(df.groupby(['laterality','run_comb','roi']))):
                        if ('bilateral' in g_name) & (g_name[2] in roi_dict):                            
                            if 'resplocwrtcentre' not in m:
                                subdf = g_df.groupby([m,'type','subid'])['neural'].mean().reset_index()
                                sns.lineplot(data=g_df,x=m,y='neural',ax=axes.flatten()[j],
                                            hue='type',palette=['blue','green'],#,'orange','red'],
                                            err_style="bars",errorbar=("se", 1),
                                            markers=True,alpha=1,linewidth=2.5)
                                #sns.lineplot(data=subdf,x=m,y='neural',ax=axes.flatten()[j],estimator=None,units="subid",
                                #            hue='type',palette=['blue','green'],#,'orange','red'],
                                #            alpha=0.2,linewidth = 1)
                            else:
                                type_colours = dict(zip(np.unique(g_df['type']),['blue','green']))
                                groups = g_df.groupby(["type","subid"])[[m,"neural"]]
                                mean, std = groups.transform("mean"), groups.transform("std")
                                normalized = (g_df[mean.columns] - mean) / std
                                g_df[mean.columns] = normalized
                                sns.regplot(data=g_df[g_df['type']=="Generalizer"],
                                            x=m,y='neural',ax=axes.flatten()[j],
                                            #scatter_kws={'alpha':0.01},
                                            units="subid",
                                            scatter=False,
                                            #ci=None,
                                            line_kws={'alpha':0.1},
                                            color=type_colours["Generalizer"])
                                sns.regplot(data=g_df[g_df['type']=="nonGeneralizer"],
                                            x=m,y='neural',ax=axes.flatten()[j],
                                            #scatter_kws={'alpha':0.01},
                                            units="subid",
                                            scatter=False,
                                            #ci=None,
                                            line_kws={'alpha':0.1},
                                            color=type_colours["nonGeneralizer"])
                            
                            axes.flatten()[j].set_title(g_name[2])
                            axes.flatten()[j].set(adjustable="datalim")
                            j += 1
                    
                    lines_labels = axes.flatten()[0].get_legend_handles_labels()
                    [ax.get_legend().remove() for ax in fig.axes if ax.get_legend() is not None]
                    lines, labels = [sum(lol, []) for lol in zip(lines_labels)]
                    #select unique legends
                    lgd_idx = [np.where(np.array(labels)==x)[0][0] for x in np.unique(labels)]
                    lines, labels = np.array(lines)[lgd_idx],np.array(labels)[lgd_idx]
                    lgd = fig.legend(lines, labels,loc = 'lower center',bbox_to_anchor=(0.5,1), ncol=6)
                    
                    grid = plt.GridSpec(prows, pcols)
                    create_subtitle(fig, grid[0, ::], 'between run')
                    create_subtitle(fig, grid[1, ::], 'within run')
                    fig.set_facecolor('w')
                    
                    fig.suptitle(fig_suptitle)
                    fig.tight_layout()
                    checkdir(os.path.join(ROIRSA_output_path,'pair-wise dissimilarity plot'))
                    fig.savefig(os.path.join(ROIRSA_output_path,'pair-wise dissimilarity plot',f'{p}_{ds_name}_{vselect}_{m}.png'),
                                bbox_inches='tight')
                    

In [8]:
g_df

,Unnamed: 0,stimidA,stimidB,groupA,groupB,runA,runB,neural,feature2d,subid,roi,laterality,voxselect,preprocess,ds,type,run_comb,group_comb
6255000,0,0,0,0,0,0,0,0.474638,NaN,sub001,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
6255001,1,0,0,0,0,0,0,0.474638,NaN,sub001,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
6255002,2,0,0,0,0,0,0,0.474638,NaN,sub001,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
6255003,3,0,0,0,0,0,0,0.474638,NaN,sub001,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
6255004,4,0,0,0,0,0,0,0.474638,NaN,sub001,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7505995,41695,24,24,0,0,24,24,0.796076,NaN,sub030,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
7505996,41696,24,24,0,0,24,24,0.796076,NaN,sub030,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
7505997,41697,24,24,0,0,24,24,0.796076,NaN,sub030,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
7505998,41698,24,24,0,0,24,24,0.796076,NaN,sub030,global - Calcarine,bilateral,no_selection,unsmoothedLSA,fourruns,Generalizer,within run,within group
